In [1]:
# Get news title and paragraph text

In [2]:
# Dependencies
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd

In [3]:
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager

In [4]:
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)



====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Corey Anderson\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


<h2> Step 1: Scraping </h2>
<h3> NASA Mars News </h3>
<p> Get latest title and news story.</p>

In [5]:
mars_url = "https://redplanetscience.com/"

In [6]:
browser.visit(mars_url)

In [7]:
html = browser.html
soup = bs(html, 'html.parser')

In [8]:
latest_titles = soup.find_all("div", class_ = "content_title")

In [9]:
print(latest_titles)

[<div class="content_title">NASA Updates Mars 2020 Mission Environmental Review</div>, <div class="content_title">NASA's Mars 2020 Rover Completes Its First Drive</div>, <div class="content_title">Virginia Middle School Student Earns Honor of Naming NASA's Next Mars Rover</div>, <div class="content_title">NASA's Perseverance Rover 100 Days Out</div>, <div class="content_title">What's Mars Solar Conjunction, and Why Does It Matter?</div>, <div class="content_title">Celebrate Mars Reconnaissance Orbiter's Views From Above</div>, <div class="content_title">NASA's Curiosity Mars Rover Takes a New Selfie Before Record Climb</div>, <div class="content_title">Global Storms on Mars Launch Dust Towers Into the Sky</div>, <div class="content_title">NASA's Perseverance Rover Will Peer Beneath Mars' Surface </div>, <div class="content_title">NASA Establishes Board to Initially Review Mars Sample Return Plans</div>, <div class="content_title">Meet the People Behind NASA's Perseverance Rover</div>, 

In [10]:
news_title = latest_titles[0].text
print(news_title)

NASA Updates Mars 2020 Mission Environmental Review


In [11]:
latest_p = soup.find_all("div", class_ = "article_teaser_body")
news_p  = latest_p[0].text
print(news_p)

NASA and the Department of Energy have completed a more detailed risk analysis for the Mars 2020 rover launch from Florida.


<h3> JPL Mars Space Images - Featured Image </h3>

In [12]:
pic_url = "https://spaceimages-mars.com/#"
browser.visit(pic_url)
html = browser.html
soup = bs(html, 'html.parser')

In [13]:
# feature_image_url

test = soup.find_all("img", class_ ="headerimage fade-in")
print(test)

url_end = test[0]["src"]
feature_image_url = f"https://spaceimages-mars.com/{url_end}"

print(feature_image_url)

[<img class="headerimage fade-in" src="image/featured/mars2.jpg"/>]
https://spaceimages-mars.com/image/featured/mars2.jpg


<h3> Mars Facts </h3>.

In [14]:
space_url = 'https://galaxyfacts-mars.com/'

In [15]:
browser.visit(space_url)
html = browser.html
soup = bs(html, 'html.parser')

In [16]:
mars_table = soup.find("table") # If you want the raw table from the site

In [17]:
# Now using pandas
easy_table = pd.read_html(space_url, header = 0)
mars_df = easy_table[0]
mars_df.set_index("Mars - Earth Comparison" , drop = True, inplace = True)

In [18]:
display(mars_df)

,Mars,Earth
Mars - Earth Comparison,,
Diameter:,"6,779 km","12,742 km"
Mass:,6.39 × 10^23 kg,5.97 × 10^24 kg
Moons:,2,1
Distance from Sun:,"227,943,824 km","149,598,262 km"
Length of Year:,687 Earth days,365.24 days
Temperature:,-87 to -5 °C,-88 to 58°C


In [19]:
table_html = mars_df.to_html()

In [20]:
soup = bs(table_html, 'html.parser')

In [21]:
t_head_only = soup.find("thead")
t_body_only = soup.find("tbody")

In [22]:
print(table_html)

<table border="1" class="dataframe">
  <thead>
    <tr style="text-align: right;">
      <th></th>
      <th>Mars</th>
      <th>Earth</th>
    </tr>
    <tr>
      <th>Mars - Earth Comparison</th>
      <th></th>
      <th></th>
    </tr>
  </thead>
  <tbody>
    <tr>
      <th>Diameter:</th>
      <td>6,779 km</td>
      <td>12,742 km</td>
    </tr>
    <tr>
      <th>Mass:</th>
      <td>6.39 × 10^23 kg</td>
      <td>5.97 × 10^24 kg</td>
    </tr>
    <tr>
      <th>Moons:</th>
      <td>2</td>
      <td>1</td>
    </tr>
    <tr>
      <th>Distance from Sun:</th>
      <td>227,943,824 km</td>
      <td>149,598,262 km</td>
    </tr>
    <tr>
      <th>Length of Year:</th>
      <td>687 Earth days</td>
      <td>365.24 days</td>
    </tr>
    <tr>
      <th>Temperature:</th>
      <td>-87 to -5 °C</td>
      <td>-88 to 58°C</td>
    </tr>
  </tbody>
</table>


<h3> Mars Hemispheres </h3>

<p> Gather image urls.</p>

In [23]:
hemi_url = "https://marshemispheres.com/"

In [24]:
browser.visit(hemi_url)
html = browser.html
soup = bs(html, 'html.parser')

In [25]:
soup

<html lang="en"><head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<link href="css/jquery-ui.css" rel="stylesheet" type="text/css"/>
<title>Astropedia Search Results | GUSS Astrogeology Science Center</title>
<meta content="GUSS Astrogeology Science Center Astropedia search results." name="description"/>
<meta content="GUSS,Astrogeology Science Center,Cartography,Geology,Space,Geological Survey,Mapping" name="keywords"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="width=device-width, initial-scale=1, maximum-scale=1" name="viewport"/>
<link href="css/main.css" media="screen" rel="stylesheet"/>
<link href="css/print.css" media="print" rel="stylesheet"/>
<link href="#" rel="icon" type="image/x-ico"/>
</head>
<body id="results">
<header>
<a href="#" style="float:right;margin-top:10px;" target="_blank">
<img alt="USGS: Science for a Changing World" class="logo" height="60" src="images/usgs_logo_main_2x.png"/>
</a>
<a href="#" style="float:

In [26]:
descriptions = soup.find_all("div", class_ = "description")

In [27]:
urls = []

for i in range(len(descriptions)):
    piece = descriptions[i].a["href"]
    urls.append(f"https://marshemishperes.com/{piece}")
    
print(urls)

['https://marshemishperes.com/cerberus.html', 'https://marshemishperes.com/schiaparelli.html', 'https://marshemishperes.com/syrtis.html', 'https://marshemishperes.com/valles.html']


In [28]:
pic_names = []

for j in range(len(descriptions)):
    pic_names.append(descriptions[j].h3.text.strip())

print(pic_names)

['Cerberus Hemisphere Enhanced', 'Schiaparelli Hemisphere Enhanced', 'Syrtis Major Hemisphere Enhanced', 'Valles Marineris Hemisphere Enhanced']


In [29]:
hemi = [None] * 4

In [30]:
for k in range(len(pic_names)):
    hemi[k] = {"title":pic_names[k], "img_url":urls[k]}

print(hemi)

[{'title': 'Cerberus Hemisphere Enhanced', 'img_url': 'https://marshemishperes.com/cerberus.html'}, {'title': 'Schiaparelli Hemisphere Enhanced', 'img_url': 'https://marshemishperes.com/schiaparelli.html'}, {'title': 'Syrtis Major Hemisphere Enhanced', 'img_url': 'https://marshemishperes.com/syrtis.html'}, {'title': 'Valles Marineris Hemisphere Enhanced', 'img_url': 'https://marshemishperes.com/valles.html'}]


In [86]:
browser.quit()

In [52]:
from splinter import Browser
from bs4 import BeautifulSoup as bs
from webdriver_manager.chrome import ChromeDriverManager

import pandas as pd



# Set up Splinter
executable_path = {'executable_path': ChromeDriverManager().install()}
browser = Browser('chrome', **executable_path, headless=False)

output_dict = {}

# Mars news_title and story
mars_url = "https://redplanetscience.com/"
browser.visit(mars_url)
html = browser.html
soup = bs(html, 'html.parser')

latest_titles = soup.find_all("div", class_ = "content_title")
new_title = latest_titles[0].text
latest_p = soup.find_all("div", class_ = "article_teaser_body")
news_p = latest_p[0].text

output_dict["title"] = new_title
output_dict["news"] = news_p

# Scrape featured image
pic_url = "https://spaceimages-mars.com/#"
browser.visit(pic_url)
html = browser.html
soup = bs(html, 'html.parser')

test = soup.find_all("img", class_ ="headerimage fade-in")
url_end = test[0]["src"]
feature_image_url = f"https://spaceimages-mars.com/{url_end}"

output_dict["image"] = feature_image_url


# Scrape table
space_url = 'https://galaxyfacts-mars.com/'
easy_table = pd.read_html(space_url, header = 0)
mars_df = easy_table[0]
mars_df.set_index("Mars - Earth Comparison" , drop = True, inplace = True)
table_html = mars_df.to_html()

output_dict["table"] = table_html


# Scrape hemisphere pics

hemi_url = "https://marshemispheres.com/"
browser.visit(hemi_url)
html = browser.html
soup = bs(html, 'html.parser')

descriptions = soup.find_all("div", class_ = "description")

pic_names = []

for j in range(len(descriptions)):
    pic_names.append(descriptions[j].h3.text.strip())

urls = []

for i in range(len(descriptions)):
    piece = descriptions[i].a["href"]
    urls.append(f"https://marshemishperes.com/{piece}")

hemi = [None] * 4 

for k in range(len(pic_names)):
    hemi[k] = {"title":pic_names[k], "img_url":urls[k]}

output_dict["hemi"] = hemi





====== WebDriver manager ======
Current google-chrome version is 91.0.4472
Get LATEST driver version for 91.0.4472
Driver [C:\Users\Corey Anderson\.wdm\drivers\chromedriver\win32\91.0.4472.101\chromedriver.exe] found in cache


In [53]:
print(output_dict)

{'title': 'NASA, ULA Launch Mars 2020 Perseverance Rover Mission to Red Planet', 'news': "The agency's Mars 2020 mission is on its way. It will land at Jezero Crater in about seven months, on Feb. 18, 2021. ", 'image': 'https://spaceimages-mars.com/image/featured/mars3.jpg', 'table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n      <td>149,598,262 km</td>\n    </tr>\n    <tr>\n      <th>Length of Year:<

In [54]:
import pymongo

In [55]:
conn = "mongodb://localhost:27017"
client = pymongo.MongoClient(conn)

In [56]:
db = client.mars_db
db.mars.insert_one(output_dict)

In [57]:
mars_output = list(db.mars.find())

In [58]:
print(mars_output)

[{'_id': ObjectId('60c672c728200535f099b9fa'), 'title': "NASA's Perseverance Rover Bringing 3D-Printed Metal Parts to Mars", 'news': "For hobbyists and makers, 3D printing expands creative possibilities; for specialized engineers, it's also key to next-generation spacecraft design.", 'image': 'https://spaceimages-mars.com/image/featured/mars1.jpg', 'table': '<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>Mars</th>\n      <th>Earth</th>\n    </tr>\n    <tr>\n      <th>Mars - Earth Comparison</th>\n      <th></th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Diameter:</th>\n      <td>6,779 km</td>\n      <td>12,742 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg</td>\n      <td>5.97 × 10^24 kg</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2</td>\n      <td>1</td>\n    </tr>\n    <tr>\n      <th>Distance from Sun:</th>\n      <td>227,943,824 km</td>\n   

In [59]:
list(db.mars.find())[0]["hemi"]

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://marshemishperes.com/cerberus.html'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://marshemishperes.com/schiaparelli.html'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://marshemishperes.com/syrtis.html'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://marshemishperes.com/valles.html'}]